## 1. 스타벅스 공식 홈페이지에서 서울 매장 정보 가져오기
### [매장명, 위도, 경도, 매장타입, 주소, 전화번호]

In [ ]:
# 라이브러리

In [1]:
# 예제 5-1 chromedriver 열기
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()


In [2]:
# webdriver 설치 및 실행 후 스타벅스 지역 매장 검색화면 접속하기, 크롬 드라이버 가져오기

# 크롬 드라이버 설치 오류시, https://chromedriver.chromium.org/home 에서 다운받고, 압축 푼 파일 경로 입력
# 스타벅스 매장 정보 URL 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
URL = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
driver.get(URL)

In [3]:
# webdriver로 '서울' 버튼 요소 찾아서 클릭하기
seoul_btn = '.set_sido_cd_btn'
driver.find_element_by_css_selector(seoul_btn).click()

/tmp/ipykernel_4597/1194876129.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(seoul_btn).click()


In [4]:
# webdriver로 '전체' 버튼 요소 찾아서 클릭하기
all_btn = '.set_gugun_cd_btn'
driver.find_element_by_css_selector(all_btn).click()

/tmp/ipykernel_4597/2028226912.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(all_btn).click()


In [5]:
# BeautifulSoup 으로 HTML 파서 만들기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# select()를 이용해 원하는 HTML 태그 찾아오기, 서울에 있는 매장 개수
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

579


In [7]:
# 태그 문자열 살펴보기
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [8]:
# 스타벅스 매장 정보 샘플 확인
# 매장명, 위도, 경도, 매장타입, 주소, 전화번호

startbucks_store = starbucks_soup_list[0]
name = startbucks_store.select('strong')[0].text.strip()
lat = startbucks_store['data-lat'].strip()
lng = startbucks_store['data-long'].strip()
store_type = startbucks_store.select('i')[0]['class'][0][4:]
address = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)         # 매장명
print(lat)          # 위도
print(lng)          # 경도
print(store_type)   # 매장 타입
print(address)      # 주소
print(tel)          # 전화번호

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [9]:
# 서울시 매장목록 데이터 만들기
starbucks_list = []

for i in starbucks_soup_list:
    name = i.select('strong')[0].text.strip()
    lat = i['data-lat'].strip()
    lng = i['data-long'].strip()
    store_type = i.select('i')[0]['class'][0][4:]
    address = str(i.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(i.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

    starbucks_list.append([name, lat, lng, store_type, address, tel])

In [11]:
import pandas as pd

In [12]:
# pandas 데이터 프레임 생성 ['매장명', '위도','경도','매장타입','주소','전화번호']
columns = ['매장명', '위도','경도','매장타입','주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns= columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [13]:
# 데이터 프레임 요약정보 확인
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     579 non-null    object
 1   위도      579 non-null    object
 2   경도      579 non-null    object
 3   매장타입    579 non-null    object
 4   주소      579 non-null    object
 5   전화번호    579 non-null    object
dtypes: object(6)
memory usage: 27.3+ KB


In [15]:
# 엑셀로 저장하기
seoul_starbucks_df.to_excel('seoul_starbucks_list.xlsx', index= False)
